# Reconstruction error

This notebook computes the Reconstruction Error (RE) to measure the effectiveness of an adversarial model attempting to infer the original trajectory by generating a candidate trajectory. 

##### **Imports**

In [1]:
import os
# oneDNN warning suppression TF 2.4.1
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import copy

import numpy as np
import pandas as pd
import pickle
import statistics

from scipy.stats import energy_distance, wasserstein_distance
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from math import radians
from sklearn.metrics.pairwise import haversine_distances
from fastdtw import fastdtw

from utils.data import *
from utils.plots import *
from utils.metrics import *
from models import *
from apu_trajgen import *

import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

##### **Parameters**

In [11]:
selected_dataset = "PORTO"

mean_min = [80, 170, 250, 350] # privacy contraint
mean_max = [155, 350, 450, 550] # utility contraint

# selected_dataset = "SANFRANCISCO"
# mean_min = [170, 250, 350] # privacy contraint
# mean_max = [350, 450, 550] # utility contraint

# Set the number of trajectories to process
n_trajs = 10 # 5000

### **Compute the reconstruction error (adaptive k)**

In [10]:

for mmin, mmax in zip(mean_min, mean_max):
        
    print("Processing trajectories with mean_min = " + str(mmin) + " and mean_max = " + str(mmax))
    
    normalization_ranges = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_normalization_ranges_test.pkl") # the scaler used to normalize the data
    normalization_ranges = {"min": normalization_ranges["min"][0:2], "max": normalization_ranges["max"][0:2]}
    
    Y_test = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_Y_test_adaptive_k" + "_ntrajs_" + str(n_trajs) + ".pkl")
    
    Y_pred_k = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_Y_pred_adaptive_k_mean_min" + str(mmin) 
            + "_mean_max" + str(mmax) + "_ntrajs_" + str(n_trajs) + ".pkl")
    Y_reconstructed_k = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_Y_reconstructed_adaptive_k_mean_min" + str(mmin) 
            + "_mean_max" + str(mmax) + "_ntrajs_" + str(n_trajs) + ".pkl")
    
    Y_reconstructed_k = Y_reconstructed_k[0]
    
    # Denormalize the data using the scaler or normalization ranges
    Y_test_dn = denormalize_data(dataset = Y_test, normalization_ranges = normalization_ranges)
    Y_pred_k_dn = denormalize_data(dataset = Y_pred_k, normalization_ranges = normalization_ranges)
    Y_reconstructed_k_dn = denormalize_data(dataset = Y_reconstructed_k, normalization_ranges = normalization_ranges)
    
    haversine_dists = []
    
    for traj_idx in range(n_trajs):
        
        # print("Trajectory: " + str(traj_idx))
        # Trajectory-level metrics
        # Haversine point-to-point distance
        haversine = compute_point_to_point_haversine_distances(Y_test_dn[traj_idx][:-1], Y_reconstructed_k_dn[traj_idx])
        
        haversine_dists.append(np.mean(haversine))
        
    print("Method: APU-TrajGen2 - adaptive k ")
    print("Targeted mean: ")
    print("Mean min: " + str(mmin) + " / Mean max: " + str(mmax))
    print("Mean reconstruction error:")
    print(np.mean(haversine_dists))

Processing trajectories with mean_min = 170 and mean_max = 350
Method: APU-TrajGen2 - adaptive k 
Targeted mean: 
Mean min: 170 / Mean max: 350
Mean reconstruction error:
289.5336902369211
Processing trajectories with mean_min = 250 and mean_max = 450
Method: APU-TrajGen2 - adaptive k 
Targeted mean: 
Mean min: 250 / Mean max: 450
Mean reconstruction error:
357.73140714579824
Processing trajectories with mean_min = 350 and mean_max = 550
Method: APU-TrajGen2 - adaptive k 
Targeted mean: 
Mean min: 350 / Mean max: 550
Mean reconstruction error:
677.2297082646662
